In [30]:
##Obtaining hmmer input

In [31]:
nameOfTheCluster = "BGC0001720" ##Change this line with the  BGC you are trying to use as the input
genbankFile = open (nameOfTheCluster + ".gbk")
hmmerInputFile = open ("hmmerInput.fasta","w")
sequence = ""
flag = False
for line in genbankFile:
    line = line.strip("\n")
    line = line.replace(" ","")
    if flag == True:
        sequence += line.strip('"')
        if line[-1] == '"':
            flag =False ##A CDS is found and the flag is turned of, now it looks for the CDS sequence
            hmmerInputFile.write(">" + nameOfTheCluster + "\n" + sequence + "\n")
            sequence = ""
    elif line[0:12] == "/translation": ##Every time it founds this, the sequence is added to the output file and the flag for looking for new CDS is turned on again
        sequence += line [14:]
        flag = True
genbankFile.close()
hmmerInputFile.close()

In [32]:
##Obtaining hmmer output. This is commented out if the user does not have hmmer. The hmmer output for BGC 1720 is provided in github
import os
#os.system("hmmscan Pfam-A.hmm hmmerinput.fasta > hmmeroutput.txt")

In [33]:
##Creating pfam counts
inputFile = open ("hmmeroutput.txt","r")
foundPfams=[] ##This list contains the pfam domains found in the BGC
for line in inputFile:
     if line[:3]==">> ":
            domainName=line[3:].split(" ")[0]
            if domainName not in foundPfams:
                foundPfams.append(domainName)

In [34]:
allPfamDomains=[] ##This list contains all the pfam domains used in the training of the model (6159)
pfamDomainsFile = open ("listOfDomains.txt")
for line in pfamDomainsFile:
    allPfamDomains.append(line.strip("\n"))

In [35]:
pfamVector = "" ##Assigns 1s to Pfams founds in the BGC and 0 to Pfams not found in the BGC
for domain in allPfamDomains:
    if domain in foundPfams:
        pfamVector += "1"
    else:
        pfamVector += "0"

In [36]:
##obtainingFingerprint
from rdkit import Chem ##These lines parse all the substrucutres in the smart file and also creates a molecule from the smiles included in the json file of the BGC
smartFile = open ("smartMolecules.tsv")
smartStringList=[]
smartMoleculeList=[]
for line in smartFile:
    line=line.strip("\n")
    line=line.split("\t")
    smartStringList.append(line[2])
smartFile.close()
for smartString in smartStringList:
    m=Chem.MolFromSmarts(smartString)
    smartMoleculeList.append(m)
smileFile = open (nameOfTheCluster + ".json")
for line in smileFile:
    line=line.strip()
    if line[0:14]=='"chem_struct":':
            smileMolecule= Chem.MolFromSmiles(line [16:-2])
smileFile.close()

In [37]:
fingerprintVector = "" ##Creates the fingerprint, 1s for substructures found in the smile string and 0 for substructures not found. The length is 306
for i in range (0, len(smartMoleculeList)):
    if (smileMolecule.HasSubstructMatch(smartMoleculeList[i])):
        fingerprintVector += "1"
    else:
        fingerprintVector += "0"

In [38]:
##IOKR

In [39]:
import numpy as np ##Trial pfam and trial fingerprint contains the correct input and output for our new BGC
trialPfam = pfamVector
trialFingerprint = fingerprintVector

In [40]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String and returns a numpy array
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [41]:
testIndexesForIOKR = np.load("testSetIndexesForTrainedModelIOKR.npy") ##Loads the files from the previously trained algorithm
trainingIndexesForIOKR = np.load("trainingSetIndexesForTrainedModelIOKR.npy")
inverseMatrixForIOKR = np.load("inverseMatrixForTrainedModelIOKR.npy")
dimensionTrainingSet = len(trainingIndexesForIOKR)
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))
trialPfam = convertStringIntoVector(trialPfam)
trialFingerprint = convertStringIntoVector(trialFingerprint)

In [42]:
##The gammas of both kernels were optimised for using IOKR with pfam counts
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two pfam vectors
    gamma=0.003
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue
def gaussianKernelForFingerprint (vector1, vector2):##Does the gaussian kernel for two fingerprint vectors. 
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [43]:
pfamTest = [pfamVectorList[i] for i in testIndexesForIOKR]##Recovers the test set from when the model was trained
pfamTest.append(trialPfam)##and adds the new file Pfam vector
fingerprintTest = [fingerprintVectorList[i] for i in testIndexesForIOKR]##Does the same for the fingerprints
fingerprintTest.append(trialFingerprint)

In [44]:
def calculatingIOKRScore (j): ##Gives the score of the IOKR formula depending on the position in the list of both pfam vectors and fingerprint vectors. Then J=206, it is the one from the new file.
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=trainingIndexesForIOKR[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],pfamTest[len(pfamTest)-1])
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],fingerprintTest[j])
    kyt=np.transpose(ky)##Transposing ky
    ##Inverse matrix was loaded from a file some cells ago. As that part comes from the original training of the algorithm.
    result= np.matmul(np.matmul(kyt,inverseMatrixForIOKR), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [45]:
score = np.zeros(207) ##Gives a score for each training of the test set. When J=206, it is the expected output vector we have to rank
for j in range (0, 207):
    score[j]= calculatingIOKRScore(j)

In [46]:
selfScore = score[len(score)-1] ##Our trial vector is the last on the list
pos = 1 ##The initial position is first, for each other vector with a better score than ours, the position will be decreased one unit. If another vector gets the same score, it is not decreased as probably it has the same fingerprint than ours.
for i in range (0, len(score)):
    if score[i] >selfScore:
        pos +=1
print ("The correct output has been ranked Top %d" % pos)

The correct output has been ranked Top 1


In [47]:
## np.argsort gives us the index of results ordered from the lowest score to the highest one.
print ("Down below are shown the top 3 fingerprint predictions:\n")
for i in [-1,-2,-3]:
    print (fingerprintTest[np.argsort (score)[i]])
    print ("\n")

Down below are shown the top 3 fingerprint predictions:

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 1 1 1 1 0 0 0 1]


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

In [48]:
##Neural network

In [49]:
numberOfClusters = len(pfamVectorList) ##This will be 1031, but this script can be used with other models already trained with more BGCs
pfamVectorList=np.concatenate(pfamVectorList,axis=0)
xData=pfamVectorList.reshape(numberOfClusters,6159)
fingerprintVectorList=np.concatenate(fingerprintVectorList,axis=0)
yData=fingerprintVectorList.reshape(numberOfClusters,306)
#The reshape puts the data as one row for each training example. It is necessary for keras
testIndexesForNN = np.load("testSetIndexesForTrainedModelNN.npy")
xTestData = [xData[i] for i in testIndexesForNN] ##The same test data than in the training is used
yTestData = [yData[i] for i in testIndexesForNN]

In [50]:
from keras.models import Sequential
from keras.models import load_model
model = load_model('neuralnetworkmodel.h5')##This file could not be uploaded to github because it is around 500 mb. If you want to try this model you should train it yourself and save the model already trained with "model.save(neuralnetworkmodel)"
trialPfam =trialPfam.reshape(1,6159)##Important to give the input for the prediction as the matrix even if using only one vector
predictedFingerprint = model.predict(trialPfam)##Gives you the predicted fingerprint for that input with the model
for j in range (0, len (predictedFingerprint[0])):##Rounds up and down all the predictions to 1 and 0 to obtain the fingerprint
        if predictedFingerprint[0][j]<0.5:
            predictedFingerprint[0][j] = 0
        else:
            predictedFingerprint[0][j] = 1

Using TensorFlow backend.
W0807 14:09:05.679498 10760 deprecation_wrapper.py:119] From C:\Users\Yoo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 14:09:05.756479 10760 deprecation_wrapper.py:119] From C:\Users\Yoo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0807 14:09:06.238243 10760 deprecation_wrapper.py:119] From C:\Users\Yoo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0807 14:09:06.239239 10760 deprecation_wrapper.py:119] From C:\Users\Yoo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0807 14:09:06.240238 10760 deprecation_wrapper.py:

In [51]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue
score = np.zeros(207)
score[206]=gaussianKernel(predictedFingerprint,trialFingerprint) ##This is the one with the correct output from the file
for j in range (0, 206):
    score[j]= gaussianKernel(predictedFingerprint,yTestData[j]) ##This are the scores with the test vectors

In [52]:
selfScore = score[len(score)-1] ##Our trial vector is the last on the list
pos = 1##The initial position is first, for each other vector with a better score than ours, the position will be decreased one unit. If another vector gets the same score, it is not decreased as probably it has the same fingerprint than ours.
for i in range (0, len(score)):
    if score[i] >selfScore:
        pos +=1
print ("The correct output has been ranked Top %d" % pos)

The correct output has been ranked Top 1


In [53]:
## np.argsort gives us the index of results ordered from the lowest score to the highest one.
print ("Down below are shown the top 3 fingerprint predictions:\n")
for i in [-1,-2,-3]:
    if np.argsort (score)[i] == 206:
        print (trialFingerprint)
    else:
        print (yTestData[np.argsort (score)[i]])
    print ("\n")

Down below are shown the top 3 fingerprint predictions:

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 1 1 1 1 0 0 0 1]


[1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 